## Objective
Build an end-to-end NLP pipeline to preprocess raw Arabic text using regex and standard NLP techniques,
then apply machine learning for sentiment classification.

# Install required libraries and Imports


In [ ]:
import kagglehub
# Download Arabic Data
path = kagglehub.dataset_download("mksaad/arabic-sentiment-twitter-corpus")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'arabic-sentiment-twitter-corpus' dataset.
Path to dataset files: /kaggle/input/arabic-sentiment-twitter-corpus


In [ ]:
!mkdir /content/Data
!cp -r {path} /content/Data

In [ ]:
!pip install --upgrade httpx
!pip install emoji -q
!pip install kagglehub -q
!pip -q install nltk
!pip install farasapy

# Install googletrans, which should automatically install httpx==0.13.3
!pip install googletrans==3.1.0a0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 3.1.0a0 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import emoji

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from farasa.segmenter import FarasaSegmenter
from nltk.stem.isri import ISRIStemmer


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Download & Load Dataset

In [ ]:
pos_train = pd.read_csv('/content/Data/arabic-sentiment-twitter-corpus/train_Arabic_tweets_positive_20190413.tsv',sep='\t',encoding='utf-8',names = ["label", "text"])
pos_test = pd.read_csv('/content/Data/arabic-sentiment-twitter-corpus/test_Arabic_tweets_positive_20190413.tsv',sep='\t',encoding='utf-8',names = ["label", "text"])

neg_train = pd.read_csv('/content/Data/arabic-sentiment-twitter-corpus/train_Arabic_tweets_negative_20190413.tsv',sep='\t',encoding='utf-8',names = ["label", "text"])
neg_test = pd.read_csv('/content/Data/arabic-sentiment-twitter-corpus/test_Arabic_tweets_negative_20190413.tsv',sep='\t',encoding='utf-8',names = ["label", "text"])

train = pd.concat([pos_train,neg_train])
test = pd.concat([pos_test,neg_test])

In [ ]:
train

,label,text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...
...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510,neg,احسدك على الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔


In [ ]:
test

,label,text
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...
1,pos,صباحك خيرات ومسرات 🌸
2,pos,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...
...,...,...
5763,neg,النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...
5764,neg,استشهاد_الامام_كاظم_الغيظ السلام على المعذب في...
5765,neg,انا كنت اكل الصحن بكبره 😐
5766,neg,قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...


In [ ]:
train['text']

,text
0,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
2,من الخير نفسه 💛
3,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...
...,...
22509,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510,احسدك على الايم 💔
22511,لأول مرة ما بنكون سوا 💔
22512,بقله ليش يا واطي 🤔


In [ ]:
test['text'][0]

,text
0,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...
0,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭


# Basic Text Cleaning

**Text** Normalization for Arabic



In [ ]:
import re

def normalize_arabic(text):
  # Remove diacritics
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text) #
  # Normalize letters variations
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ڤ", "ف", text)
    text = re.sub("چ", "ج", text)
    text = re.sub("پ", "ب", text)
    text = re.sub("ڜ", "ش", text)
    text = re.sub("ڪ", "ك", text)
    text = re.sub("ڧ", "ق", text)
    text = re.sub("ٱ", "ا", text)
    return text

In [ ]:
train["clean_text"] = train["text"].apply(normalize_arabic)
test["clean_text"] = test["text"].apply(normalize_arabic)

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,نحن الذين يتحول كل ما نود ان نقوله الي دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي النهايه لن يبقي معك احدالا من راي الجمال ف...
2,pos,من الخير نفسه 💛,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,الشيء الوحيد الذي وصلوا فيه للعالميه هو : المس...
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,كيف تري اورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510,neg,احسدك على الايم 💔,احسدك علي الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔,لاول مره ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔,بقله ليش يا واطي 🤔


**Arabic-Specific Noise Removal**


In [ ]:
def remove_arabic_noise(text):
    # Remove tatweel
    text = re.sub(r'\u0640', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
train["clean_text"] = train["clean_text"].apply(remove_arabic_noise)
test["clean_text"] = test["clean_text"].apply(remove_arabic_noise)

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,نحن الذين يتحول كل ما نود ان نقوله الي دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي النهايه لن يبقي معك احدالا من راي الجمال ف...
2,pos,من الخير نفسه 💛,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,الشيء الوحيد الذي وصلوا فيه للعالميه هو : المس...
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,كيف تري اورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510,neg,احسدك على الايم 💔,احسدك علي الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔,لاول مره ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔,بقله ليش يا واطي 🤔


**Remove Puncatuations**

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""

all_punctuations = arabic_punctuations + english_punctuations

def remove_arabic_punctuations(text):
    return re.sub(f"[{re.escape(all_punctuations)}]", " ", text)


In [ ]:
train["clean_text"] = train["clean_text"].apply(remove_arabic_punctuations)
test["clean_text"] = test["clean_text"].apply(remove_arabic_punctuations)

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,نحن الذين يتحول كل ما نود ان نقوله الي دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي النهايه لن يبقي معك احدالا من راي الجمال ف...
2,pos,من الخير نفسه 💛,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,زلزل الملعب نصرنا بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,الشيء الوحيد الذي وصلوا فيه للعالميه هو المس...
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,كيف تري اورانوس لو كان يقع مكان القمر 💙💙 كوك...
22510,neg,احسدك على الايم 💔,احسدك علي الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔,لاول مره ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔,بقله ليش يا واطي 🤔


 **Correct Arabic Text**

In [ ]:
def correct_arabic_text(text):

    corrections = {
        'انشاء الله': 'إن شاء الله',
        'انشالله': 'إن شاء الله',
        'ان شا الله': 'إن شاء الله',
        'الله يرحمه': 'الله يرحمه',
        'يارب': 'يا رب',
        'ياارب': 'يا رب',

        'لاكن': 'لكن',
        'هاذا': 'هذا',
        'مشاءالله': 'ما شاء الله',
        'اللهم امين': 'اللهم آمين',
        'ءن': 'إن',

        'يارب': 'يا رب',
        'يااارب': 'يا رب',
        'وينك': 'أين أنت',
        'وينهم': 'أين هم',

        'هههههه': 'هه',
        'خخخخخ': 'خ',

        'مش': 'ليس',
        'اووي': 'جدا',
        'اوي': 'جدا',
        'كده': 'كذا',
        'ليه': 'لماذا',
        'عايز': 'أريد',

        'الذي': 'الذي',
        'اللذي': 'الذي',

        'احسن': 'أفضل',
        'كويس': 'جيد',
        'جداا': 'جدا',
    }

    for mistake, correction in corrections.items():
        text = text.replace(mistake, correction)

    return text


In [ ]:
train["clean_text"] = train["clean_text"].apply(correct_arabic_text)
test["clean_text"] = test["clean_text"].apply(correct_arabic_text)

**Normalize Nmbers**

In [ ]:
def handle_numbers_and_special_chars(text, mode='remove'):
    if mode == 'remove':
        text = re.sub(r'\d+', '', text)
        return text
    elif mode == 'normalize':
        number_map = {
                      '٠': 'صفر',
                      '١': 'واحد',
                      '٢': 'اثنان',
                      '٣': 'ثلاثة',
                      '٤': 'أربعة',
                      '٥': 'خمسة',
                      '٦': 'ستة',
                      '٧': 'سبعة',
                      '٨': 'ثمانية',
                      '٩': 'تسعة',
                      '0': 'صفر',
                      '1': 'واحد',
                      '2': 'اثنان',
                      '3': 'ثلاثة',
                      '4': 'أربعة',
                      '5': 'خمسة',
                      '6': 'ستة',
                      '7': 'سبعة',
                      '8': 'ثمانية',
                      '9': 'تسعة'
                  }
        for arabic, name in number_map.items():
            text = text.replace(arabic, name)
        return text

In [ ]:
train["clean_text"] = train["clean_text"].apply(handle_numbers_and_special_chars,mode='normalize')
test["clean_text"] = test["clean_text"].apply(handle_numbers_and_special_chars,mode='normalize')

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,نحن الذين يتحول كل ما نود ان نقوله الي دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي النهايه لن يبقي معك احدالا من راي الجمال ف...
2,pos,من الخير نفسه 💛,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,زلزل الملعب نصرنا بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,الشيء الوحيد الذي وصلوا فيه للعالميه هو المس...
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,كيف تري اورانوس لو كان يقع مكان القمر 💙💙 كوك...
22510,neg,احسدك على الايم 💔,احسدك علي الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔,لاول مره ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔,بقله ليش يا واطي 🤔


**Normalize Letters and Handel franco**

In [ ]:
import re

def arabizi_to_arabic(text):
    text = text.lower()

    word_map = {
        "enta": "انت",
        "enti": "انتي",
        "ana": "انا",
        "la2": "لا",
        "mashy": "ماشي",
        "leeh": "ليه",
        "le": "لي",
        "akl": "اكل",
        "sa7": "صح",
        "akid": "اكيد",
        "momken": "ممكن",
        "haga": "حاجه",
        "a7la": "احلى",
        "a7bab": "احباب",
        "ba2a": "بقى",
        "3shan": "عشان",
        "3alashan": "علشان",
        "7elwa": "حلوه",
        "7elw": "حلو",
        "7abibi": "حبيبي",
        "7abibti": "حبيبتي",
        "b7ebak": "بحبك",
        "b7ebek": "بحبك",
        "b7bk": "بحبك",
    }

    for w, a in word_map.items():
        text = re.sub(rf"\b{w}\b", a, text)

    text = text.replace("sh", "ش")
    text = text.replace("ch", "تش")
    text = text.replace("kh", "خ")
    text = text.replace("gh", "غ")

    num_map = {
        "2": "ء",
        "3": "ع",
        "4": "غ",
        "5": "خ",
        "6": "ط",
        "6'": "ظ",
        "7": "ح",
        "8": "ق",
        "9": "ص",
        "9'": "ض"
    }

    for num, arab in num_map.items():
        text = text.replace(num, arab)

    letter_map = {
        "a": "ا", "b": "ب", "t": "ت", "g": "ج", "d": "د",
        "r": "ر", "z": "ز", "s": "س", "f": "ف", "q": "ق",
        "k": "ك", "l": "ل", "m": "م", "n": "ن", "h": "ه",
        "w": "و", "y": "ي"
    }

    for l, a in letter_map.items():
        text = text.replace(l, a)

    return text


In [ ]:
train["clean_text"] = train["clean_text"].apply(arabizi_to_arabic)
test["clean_text"] = test["clean_text"].apply(arabizi_to_arabic)

**Handling Emojis and Emoticons in Arabic Text**



In [ ]:
def handle_emojis(text, mode='remove'):
    if mode == 'remove':
        return emoji.replace_emoji(text, '')
    elif mode == 'description':
        return emoji.demojize(text, language='ar')
    return text

In [ ]:
train["clean_text"] = train["clean_text"].apply(handle_emojis ,mode='description')
test["clean_text"] = test["clean_text"].apply(handle_emojis,mode='description')

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,نحن الذين يتحول كل ما نود ان نقوله الي دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي النهايه لن يبقي معك احدالا من راي الجمال ف...
2,pos,من الخير نفسه 💛,من الخير نفسه :قلب_أصفر:
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,زلزل الملعب نصرنا بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,الشيء الوحيد الذي وصلوا فيه للعالميه هو المس...
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,كيف تري اورانوس لو كان يقع مكان القمر :قلب_أ...
22510,neg,احسدك على الايم 💔,احسدك علي الايم :قلب_مكسور:
22511,neg,لأول مرة ما بنكون سوا 💔,لاول مره ما بنكون سوا :قلب_مكسور:
22512,neg,بقله ليش يا واطي 🤔,بقله ليش يا واطي :وجه_يفكر:


In [ ]:
test

,label,text,clean_text
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,الهلال الاهلي فوز هلالي مهم الحمد لله :قلب_أز...
1,pos,صباحك خيرات ومسرات 🌸,صباحك خيرات ومسرات :زهرة_الكرز:
2,pos,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,تامل قال الله ﷻ ​﴿بواد غير ذي زرع ﴾ :د...
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...,:وجه_بدموع_فرح::وجه_بدموع_فرح: يا جدعان الرجال...
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...,رساله صباحيه :قلب_أصفر: اللهم اسالك التوفيق ...
...,...,...,...
5763,neg,النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...,النوم وانت مكسور ده احساس غبي اللي هو ليس قادر...
5764,neg,استشهاد_الامام_كاظم_الغيظ السلام على المعذب في...,استشهاد الامام كاظم الغيظ السلام علي المعذب في...
5765,neg,انا كنت اكل الصحن بكبره 😐,انا كنت اكل الصحن بكبره :وجه_محايد:
5766,neg,قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...,قولوا لي ايش تشوفوا مع ملاحظه التلطف لانه ا...


**Handling Elongated Words in Arabic**

In [ ]:
def normalize_elongated_words(text):
    # Remove elongation
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text


In [ ]:
train["clean_text"] = train["clean_text"].apply(normalize_elongated_words)
test["clean_text"] = test["clean_text"].apply(normalize_elongated_words)

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,نحن الذين يتحول كل ما نود ان نقوله الي دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي النهايه لن يبقي معك احدالا من راي الجمال ف...
2,pos,من الخير نفسه 💛,من الخير نفسه :قلب_أصفر:
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,زلزل الملعب نصرنا بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,الشيء الوحيد الذي وصلوا فيه للعالميه هو المسي...
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,كيف تري اورانوس لو كان يقع مكان القمر :قلب_أز...
22510,neg,احسدك على الايم 💔,احسدك علي الايم :قلب_مكسور:
22511,neg,لأول مرة ما بنكون سوا 💔,لاول مره ما بنكون سوا :قلب_مكسور:
22512,neg,بقله ليش يا واطي 🤔,بقله ليش يا واطي :وجه_يفكر:


#Tokenization

In [ ]:
train["clean_text"] = train["clean_text"].apply(word_tokenize)
test["clean_text"] = test["clean_text"].apply(word_tokenize)

In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[نحن, الذين, يتحول, كل, ما, نود, ان, نقوله, ال..."
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, النهايه, لن, يبقي, معك, احدالا, من, راي,..."
2,pos,من الخير نفسه 💛,"[من, الخير, نفسه, :, قلب_أصفر, :]"
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,"[زلزل, الملعب, نصرنا, بيلعب, كن, عالي, الهمه, ..."
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,"[الشيء, الوحيد, الذي, وصلوا, فيه, للعالميه, هو..."
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,"[كيف, تري, اورانوس, لو, كان, يقع, مكان, القمر,..."
22510,neg,احسدك على الايم 💔,"[احسدك, علي, الايم, :, قلب_مكسور, :]"
22511,neg,لأول مرة ما بنكون سوا 💔,"[لاول, مره, ما, بنكون, سوا, :, قلب_مكسور, :]"
22512,neg,بقله ليش يا واطي 🤔,"[بقله, ليش, يا, واطي, :, وجه_يفكر, :]"


#Stopword Removal

In [ ]:
arabic_stopwords = set(stopwords.words('arabic'))

# remove stopwords FROM LIST OF TOKENS
train['clean_text'] = train['clean_text'].apply(
    lambda tokens: [word for word in tokens if word not in arabic_stopwords]
)

test['clean_text'] = test['clean_text'].apply(
    lambda tokens: [word for word in tokens if word not in arabic_stopwords]
)


In [ ]:
arabic_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[يتحول, نود, ان, نقوله, الي, دعاء, لله, تبحثوا..."
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, النهايه, يبقي, معك, احدالا, راي, الجمال,..."
2,pos,من الخير نفسه 💛,"[الخير, نفسه, :, قلب_أصفر, :]"
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,"[زلزل, الملعب, نصرنا, بيلعب, عالي, الهمه, ترضي..."
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,"[الشيء, الوحيد, وصلوا, للعالميه, المسيار, تري,..."
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,"[تري, اورانوس, يقع, مكان, القمر, :, قلب_أزرق, ..."
22510,neg,احسدك على الايم 💔,"[احسدك, علي, الايم, :, قلب_مكسور, :]"
22511,neg,لأول مرة ما بنكون سوا 💔,"[لاول, مره, بنكون, سوا, :, قلب_مكسور, :]"
22512,neg,بقله ليش يا واطي 🤔,"[بقله, ليش, واطي, :, وجه_يفكر, :]"


# Stemming


In [ ]:
stemmer = ISRIStemmer()

train['clean_text'] = train['clean_text'].apply(
    lambda tokens: [stemmer.stem(word) for word in tokens]
)

test['clean_text'] = test['clean_text'].apply(
    lambda tokens: [stemmer.stem(word) for word in tokens]
)


In [ ]:
train

,label,text,clean_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق..."
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال..."
2,pos,من الخير نفسه 💛,"[خير, نفس, :, قلب_أصفر, :]"
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,"[زلزل, لعب, نصر, لعب, علي, همه, ترض, بغر, قمه,..."
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,"[شيء, وحد, وصل, علم, سير, تري, كان, شجع, درد, ..."
...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,"[تري, رنس, يقع, كان, قمر, :, قلب_أزرق, :, :قلب..."
22510,neg,احسدك على الايم 💔,"[حسد, علي, ايم, :, قلب_مكسور, :]"
22511,neg,لأول مرة ما بنكون سوا 💔,"[لول, مره, بنك, سوا, :, قلب_مكسور, :]"
22512,neg,بقله ليش يا واطي 🤔,"[بقل, ليش, وطي, :, وجه_يفكر, :]"


In [ ]:
train['clean_text_str'] = train['clean_text'].apply(lambda tokens: " ".join(tokens))
test['clean_text_str']  = test['clean_text'].apply(lambda tokens: " ".join(tokens))

In [ ]:
train['clean_text_str'].iloc[0]

'تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر قوه'

In [ ]:
test

,label,text,clean_text,clean_text_str
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,"[هلل, اهل, فوز, هلل, مهم, حمد, لله, :, قلب_أزر...",هلل اهل فوز هلل مهم حمد لله : قلب_أزرق : زور س...
1,pos,صباحك خيرات ومسرات 🌸,"[صبح, خير, مسر, :, زهرة_الكرز, :]",صبح خير مسر : زهرة_الكرز :
2,pos,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,"[تمل, قال, الل, ﷻ, ​﴿بواد, زرع, ﴾, :, دور, :, ...",تمل قال الل ﷻ ​﴿بواد زرع ﴾ : دور : :دوار : : د...
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...,"[:, وجه_بدموع_فرح, :, :وجه_بدموع_فرح, :, جدع, ...",: وجه_بدموع_فرح : :وجه_بدموع_فرح : جدع رجل الل...
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...,"[رسل, صبح, :, قلب_أصفر, :, لهم, سلك, وفق, امر,...",رسل صبح : قلب_أصفر : لهم سلك وفق امر كتب ردس و...
...,...,...,...,...
5763,neg,النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...,"[نوم, ونت, كسر, ده, حسس, غبي, الل, قدر, تنم, ل...",نوم ونت كسر ده حسس غبي الل قدر تنم لزم علش تصح...
5764,neg,استشهاد_الامام_كاظم_الغيظ السلام على المعذب في...,"[شهد, امم, كظم, غيظ, سلم, علي, عذب, قعر, سجن, ...",شهد امم كظم غيظ سلم علي عذب قعر سجن ظلم مطامير...
5765,neg,انا كنت اكل الصحن بكبره 😐,"[انا, كنت, اكل, صحن, كبر, :, وجه_محايد, :]",انا كنت اكل صحن كبر : وجه_محايد :
5766,neg,قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...,"[قول, ايش, تشف, لحظ, لطف, لنه, مود, علماذ, :, ...",قول ايش تشف لحظ لطف لنه مود علماذ : وجه_ناعس :


#English to Arabic

In [ ]:
# Firsy check if there is or not ?
def contains_english(text):
    return bool(re.search(r'[A-Za-z]', text))


In [ ]:
train['has_english'] = train['clean_text_str'].apply(contains_english)
train['has_english'].value_counts()


,count
has_english,
False,43750
True,1525


In [ ]:
test['has_english'] = test['clean_text_str'].apply(contains_english)
test['has_english'].value_counts()

,count
has_english,
False,11148
True,372


In [ ]:
def extract_english_words(text):
    return re.findall(r'[A-Za-z]+', text)

In [ ]:
train['english_words'] = train['clean_text_str'].apply(extract_english_words)

In [ ]:
test['english_words'] = test['clean_text_str'].apply(extract_english_words)

In [ ]:
train

,label,text,clean_text,clean_text_str,has_english,english_words
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق...",تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر...,False,[]
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال...",وفي نهي يبق معك حدل راي جمل روح اماالمنبهر مظا,False,[]
2,pos,من الخير نفسه 💛,"[خير, نفس, :, قلب_أصفر, :]",خير نفس : قلب_أصفر :,False,[]
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,"[زلزل, لعب, نصر, لعب, علي, همه, ترض, بغر, قمه,...",زلزل لعب نصر لعب علي همه ترض بغر قمه جرد ساع ل...,False,[]
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,"[شيء, وحد, وصل, علم, سير, تري, كان, شجع, درد, ...",شيء وحد وصل علم سير تري كان شجع درد ضد نصر : و...,False,[]
...,...,...,...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,"[تري, رنس, يقع, كان, قمر, :, قلب_أزرق, :, :قلب...",تري رنس يقع كان قمر : قلب_أزرق : :قلب_أزرق : ك...,False,[]
22510,neg,احسدك على الايم 💔,"[حسد, علي, ايم, :, قلب_مكسور, :]",حسد علي ايم : قلب_مكسور :,False,[]
22511,neg,لأول مرة ما بنكون سوا 💔,"[لول, مره, بنك, سوا, :, قلب_مكسور, :]",لول مره بنك سوا : قلب_مكسور :,False,[]
22512,neg,بقله ليش يا واطي 🤔,"[بقل, ليش, وطي, :, وجه_يفكر, :]",بقل ليش وطي : وجه_يفكر :,False,[]


In [ ]:
test

,label,text,clean_text,clean_text_str,has_english,english_words
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,"[هلل, اهل, فوز, هلل, مهم, حمد, لله, :, قلب_أزر...",هلل اهل فوز هلل مهم حمد لله : قلب_أزرق : زور س...,False,[]
1,pos,صباحك خيرات ومسرات 🌸,"[صبح, خير, مسر, :, زهرة_الكرز, :]",صبح خير مسر : زهرة_الكرز :,False,[]
2,pos,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,"[تمل, قال, الل, ﷻ, ​﴿بواد, زرع, ﴾, :, دور, :, ...",تمل قال الل ﷻ ​﴿بواد زرع ﴾ : دور : :دوار : : د...,False,[]
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...,"[:, وجه_بدموع_فرح, :, :وجه_بدموع_فرح, :, جدع, ...",: وجه_بدموع_فرح : :وجه_بدموع_فرح : جدع رجل الل...,False,[]
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...,"[رسل, صبح, :, قلب_أصفر, :, لهم, سلك, وفق, امر,...",رسل صبح : قلب_أصفر : لهم سلك وفق امر كتب ردس و...,False,[]
...,...,...,...,...,...,...
5763,neg,النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...,"[نوم, ونت, كسر, ده, حسس, غبي, الل, قدر, تنم, ل...",نوم ونت كسر ده حسس غبي الل قدر تنم لزم علش تصح...,False,[]
5764,neg,استشهاد_الامام_كاظم_الغيظ السلام على المعذب في...,"[شهد, امم, كظم, غيظ, سلم, علي, عذب, قعر, سجن, ...",شهد امم كظم غيظ سلم علي عذب قعر سجن ظلم مطامير...,False,[]
5765,neg,انا كنت اكل الصحن بكبره 😐,"[انا, كنت, اكل, صحن, كبر, :, وجه_محايد, :]",انا كنت اكل صحن كبر : وجه_محايد :,False,[]
5766,neg,قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...,"[قول, ايش, تشف, لحظ, لطف, لنه, مود, علماذ, :, ...",قول ايش تشف لحظ لطف لنه مود علماذ : وجه_ناعس :,False,[]


In [ ]:
percent = train['has_english'].mean() * 100
print(f"Percent of rows containing English: {percent:.2f}%")


Percent of rows containing English: 3.37%


In [ ]:
all_english = sum(train['english_words'], [])
Counter(all_english).most_common(30)


[('e', 1274),
 ('po', 1142),
 ('x', 623),
 ('i', 616),
 ('o', 496),
 ('u', 216),
 ('p', 130),
 ('c', 119),
 ('uv', 111),
 ('co', 56),
 ('oo', 54),
 ('pi', 50),
 ('v', 46),
 ('xco', 41),
 ('ou', 40),
 ('io', 38),
 ('pe', 25),
 ('ei', 20),
 ('ip', 19),
 ('eo', 16),
 ('ie', 13),
 ('ic', 12),
 ('ive', 11),
 ('ove', 10),
 ('ee', 10),
 ('up', 10),
 ('vi', 9),
 ('eve', 9),
 ('pu', 9),
 ('exp', 8)]

#unicode artifacts words

In [ ]:
translation_map = {
    "luv": "حب",
    "goo": "جيد",
    "be": "يكون",
    "wi": "مع",
    "fifa": "فيفا",
}

noise_words = {
    "po","e","xr","i","o","u","x","c","bo","d","a","m","p",
    "s","t","k","g","e","io","c","co","o","i","f"
}


In [ ]:
def translate_english_tokens(token_list):
    cleaned = []
    for word in token_list:
        w = word.lower()
        if w in translation_map:
            cleaned.append(translation_map[w])
        elif w in noise_words:
            continue
        else:
            cleaned.append(word)
    return cleaned


In [ ]:
# clean_text = tokens → apply translation
train["clean_translated_tokens"] = train["clean_text"].apply(translate_english_tokens)

In [ ]:
# clean_text = tokens → apply translation
train["clean_translated_tokens"] = train["clean_text"].apply(translate_english_tokens)

In [ ]:
train

,label,text,clean_text,clean_text_str,has_english,english_words,clean_translated_tokens
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق...",تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر...,False,[],"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق..."
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال...",وفي نهي يبق معك حدل راي جمل روح اماالمنبهر مظا,False,[],"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال..."
2,pos,من الخير نفسه 💛,"[خير, نفس, :, قلب_أصفر, :]",خير نفس : قلب_أصفر :,False,[],"[خير, نفس, :, قلب_أصفر, :]"
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,"[زلزل, لعب, نصر, لعب, علي, همه, ترض, بغر, قمه,...",زلزل لعب نصر لعب علي همه ترض بغر قمه جرد ساع ل...,False,[],"[زلزل, لعب, نصر, لعب, علي, همه, ترض, بغر, قمه,..."
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,"[شيء, وحد, وصل, علم, سير, تري, كان, شجع, درد, ...",شيء وحد وصل علم سير تري كان شجع درد ضد نصر : و...,False,[],"[شيء, وحد, وصل, علم, سير, تري, كان, شجع, درد, ..."
...,...,...,...,...,...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,"[تري, رنس, يقع, كان, قمر, :, قلب_أزرق, :, :قلب...",تري رنس يقع كان قمر : قلب_أزرق : :قلب_أزرق : ك...,False,[],"[تري, رنس, يقع, كان, قمر, :, قلب_أزرق, :, :قلب..."
22510,neg,احسدك على الايم 💔,"[حسد, علي, ايم, :, قلب_مكسور, :]",حسد علي ايم : قلب_مكسور :,False,[],"[حسد, علي, ايم, :, قلب_مكسور, :]"
22511,neg,لأول مرة ما بنكون سوا 💔,"[لول, مره, بنك, سوا, :, قلب_مكسور, :]",لول مره بنك سوا : قلب_مكسور :,False,[],"[لول, مره, بنك, سوا, :, قلب_مكسور, :]"
22512,neg,بقله ليش يا واطي 🤔,"[بقل, ليش, وطي, :, وجه_يفكر, :]",بقل ليش وطي : وجه_يفكر :,False,[],"[بقل, ليش, وطي, :, وجه_يفكر, :]"


In [ ]:
train["clean_translated_str"] = train["clean_translated_tokens"].apply(
    lambda tokens: " ".join(tokens)
)

In [ ]:
train.head(3)

,label,text,clean_text,clean_text_str,has_english,english_words,clean_translated_tokens,clean_translated_str
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق...",تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر...,False,[],"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق...",تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال...",وفي نهي يبق معك حدل راي جمل روح اماالمنبهر مظا,False,[],"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال...",وفي نهي يبق معك حدل راي جمل روح اماالمنبهر مظا
2,pos,من الخير نفسه 💛,"[خير, نفس, :, قلب_أصفر, :]",خير نفس : قلب_أصفر :,False,[],"[خير, نفس, :, قلب_أصفر, :]",خير نفس : قلب_أصفر :


In [ ]:
# clean_text = tokens → apply translation
train["clean_translated_tokens"] = train["clean_text"].apply(translate_english_tokens)

In [ ]:
train["clean_translated_str"] = train["clean_translated_tokens"].apply(
    lambda tokens: " ".join(tokens)
)

In [ ]:
test.head(3)

,label,text,clean_text,clean_text_str,has_english,english_words
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,"[هلل, اهل, فوز, هلل, مهم, حمد, لله, :, قلب_أزر...",هلل اهل فوز هلل مهم حمد لله : قلب_أزرق : زور س...,False,[]
1,pos,صباحك خيرات ومسرات 🌸,"[صبح, خير, مسر, :, زهرة_الكرز, :]",صبح خير مسر : زهرة_الكرز :,False,[]
2,pos,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,"[تمل, قال, الل, ﷻ, ​﴿بواد, زرع, ﴾, :, دور, :, ...",تمل قال الل ﷻ ​﴿بواد زرع ﴾ : دور : :دوار : : د...,False,[]


In [ ]:
def detect_english_tokens(tokens):
    english_found = []
    for w in tokens:
        if re.search(r"[A-Za-z]", w):
            english_found.append(w)
    return english_found

train["english_words_after_clean"] = train["clean_translated_tokens"].apply(detect_english_tokens)

train["has_english_after_clean"] = train["english_words_after_clean"].apply(
    lambda lst: len(lst) > 0
)

In [ ]:
percent_after = train["has_english_after_clean"].mean() * 100
print(f"Percent of rows still containing English AFTER cleaning: {percent_after:.3f}%")

Percent of rows still containing English AFTER cleaning: 3.176%


In [ ]:
train['label'] = train['label'].map({'pos': 1, 'neg': 0})
train.head()

,label,text,clean_text,clean_text_str,has_english,english_words,clean_translated_tokens,clean_translated_str,english_words_after_clean,has_english_after_clean
0,1,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق...",تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر...,False,[],"[تحل, نود, ان, نقل, الي, دعء, لله, بحث, فين, ق...",تحل نود ان نقل الي دعء لله بحث فين قوه انن كسر...,[],False
1,1,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال...",وفي نهي يبق معك حدل راي جمل روح اماالمنبهر مظا,False,[],"[وفي, نهي, يبق, معك, حدل, راي, جمل, روح, اماال...",وفي نهي يبق معك حدل راي جمل روح اماالمنبهر مظا,[],False
2,1,من الخير نفسه 💛,"[خير, نفس, :, قلب_أصفر, :]",خير نفس : قلب_أصفر :,False,[],"[خير, نفس, :, قلب_أصفر, :]",خير نفس : قلب_أصفر :,[],False
3,1,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,"[زلزل, لعب, نصر, لعب, علي, همه, ترض, بغر, قمه,...",زلزل لعب نصر لعب علي همه ترض بغر قمه جرد ساع ل...,False,[],"[زلزل, لعب, نصر, لعب, علي, همه, ترض, بغر, قمه,...",زلزل لعب نصر لعب علي همه ترض بغر قمه جرد ساع ل...,[],False
4,1,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,"[شيء, وحد, وصل, علم, سير, تري, كان, شجع, درد, ...",شيء وحد وصل علم سير تري كان شجع درد ضد نصر : و...,False,[],"[شيء, وحد, وصل, علم, سير, تري, كان, شجع, درد, ...",شيء وحد وصل علم سير تري كان شجع درد ضد نصر : و...,[],False


# Train & Test Split

In [ ]:
X = train['clean_text_str']
y = train['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
)

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)


# Model — Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=2000)
model.fit(X_train_vec, y_train)


LogisticRegression(max_iter=2000)

# Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9171728326891221
              precision    recall  f1-score   support

           0       0.94      0.89      0.91      4503
           1       0.90      0.94      0.92      4552

    accuracy                           0.92      9055
   macro avg       0.92      0.92      0.92      9055
weighted avg       0.92      0.92      0.92      9055

[[4026  477]
 [ 273 4279]]
